# Optimizing Quick Commerce Logistics in Kochi (Full Notebook)
This notebook fetches OpenStreetMap data for Kochi, creates demand zones, estimates demand, and runs:
- Facility location optimization (PuLP)
- Vehicle routing problem (OR-Tools)
- Visualizations (Folium)

Run all cells in Colab. Cells install dependencies automatically.

In [ ]:
# Install dependencies (runs in Colab)
!pip install osmnx==1.2.2 geopandas pulp ortools folium matplotlib pandas numpy networkx scipy scikit-learn


In [ ]:
import os
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point
from scripts.geo_utils import haversine_km
print('Libraries imported')


## 1) Fetch Kochi boundary, road network, and POIs
This may take a minute. We use OSMnx to download the city polygon, road network, and POIs.

In [ ]:
place = 'Kochi, Kerala, India'
print('Geocoding:', place)
koch = ox.geocode_to_gdf(place)
koch.to_file('data/koch_boundary.geojson', driver='GeoJSON')
print('Saved boundary')

# Road network (drive)
G = ox.graph_from_place(place, network_type='drive')
ox.save_graphml(G, 'data/koch_graph.graphml')
print('Saved graph')

# POIs (shops, supermarkets, warehouses)
tags = {'shop':True, 'supermarket':True, 'warehouse':True}
pois = ox.geometries_from_place(place, tags)
pois = pois[['geometry']].dropna()
pois.to_file('data/koch_pois.geojson', driver='GeoJSON')
print('Saved POIs:', len(pois))


## 2) Create grid demand zones (approx 2 km cells) and estimate demand


In [ ]:
from shapely.geometry import box
poly = koch.geometry.iloc[0]
minx, miny, maxx, maxy = poly.bounds
cell_size_deg = 0.018
cells = []
xs = np.arange(minx, maxx, cell_size_deg)
ys = np.arange(miny, maxy, cell_size_deg)
for x in xs:
    for y in ys:
        cell = box(x, y, x+cell_size_deg, y+cell_size_deg)
        if cell.intersects(poly):
            cells.append(cell)
zones = gpd.GeoDataFrame({'geometry': cells}, crs=koch.crs)
zones['centroid'] = zones.geometry.centroid
zones = zones.set_geometry('geometry')
zones.to_file('data/koch_zones.geojson', driver='GeoJSON')
print('Created', len(zones), 'zones')
zones['area'] = zones.geometry.area
zones['demand'] = (zones['area']/zones['area'].sum() * 2000).round().astype(int)
zones[['demand']].to_csv('data/zone_demands.csv', index=False)
print('Saved zone demands')


## 3) Candidate facilities selection (k-means on POI centroids)


In [ ]:
candidates = pois.copy()
candidates['x'] = candidates.geometry.centroid.x
candidates['y'] = candidates.geometry.centroid.y
from sklearn.cluster import KMeans
k = min(8, len(candidates))
if k>0:
    km = KMeans(n_clusters=k, random_state=0).fit(candidates[['x','y']])
    candidates['cluster'] = km.labels_
    cand = candidates.dissolve(by='cluster', aggfunc='first').reset_index(drop=True)
else:
    cand = gpd.GeoDataFrame({'geometry':[Point(minx + (i+0.5)*cell_size_deg, miny + (i+0.5)*cell_size_deg) for i in range(6)]}, crs=koch.crs)

cand = cand.head(8).copy()
cand['fixed_cost'] = [1200,1100,1300,900,1000,1400,1250,1150][:len(cand)]
cand['capacity'] = [800,700,900,600,750,1000,850,900][:len(cand)]
cand.to_file('data/candidates.geojson', driver='GeoJSON')
print('Saved', len(cand), 'candidate facilities')


## 4) Facility location MILP (PuLP)


In [ ]:
fac_coords = [(pt.y, pt.x) for pt in cand.geometry.centroid]
zone_coords = [(pt.y, pt.x) for pt in zones.geometry.centroid]
M = len(fac_coords); N = len(zone_coords)
transport_cost = np.zeros((M,N))
for i in range(M):
    for j in range(N):
        transport_cost[i,j] = haversine_km(fac_coords[i][1], fac_coords[i][0], zone_coords[j][1], zone_coords[j][0]) * 8

import pulp
fac_df = pd.DataFrame({'fixed_cost':cand['fixed_cost'].values, 'capacity':cand['capacity'].values})
zone_df = pd.DataFrame({'demand':zones['demand'].values})
prob = pulp.LpProblem('FacilityLocation', pulp.LpMinimize)
open_var = pulp.LpVariable.dicts('open', range(M), 0,1,cat='Binary')
flow = pulp.LpVariable.dicts('flow', [(i,j) for i in range(M) for j in range(N)], lowBound=0)
prob += pulp.lpSum([open_var[i]*fac_df.loc[i,'fixed_cost'] for i in range(M)]) + pulp.lpSum([flow[(i,j)]*transport_cost[i,j] for i in range(M) for j in range(N)])
for j in range(N):
    prob += pulp.lpSum([flow[(i,j)] for i in range(M)]) == float(zone_df.loc[j,'demand'])
for i in range(M):
    prob += pulp.lpSum([flow[(i,j)] for j in range(N)]) <= float(fac_df.loc[i,'capacity'])*open_var[i]
prob.solve(pulp.PULP_CBC_CMD(msg=0))
open_list = [int(open_var[i].value()) for i in range(M)]
print('Open facilities:', open_list)

pd.DataFrame({'open':open_list}).to_csv('data/open_facilities.csv', index=False)
pd.DataFrame(transport_cost).to_csv('data/transport_cost_matrix.csv', index=False)
print('Saved outputs')


## 5) VRP on top-demand zones (OR-Tools)


In [ ]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
open_idx = None
for i,o in enumerate(open_list):
    if o==1:
        open_idx = i
        break
if open_idx is None:
    open_idx = 0

zones_small = zones.copy()
zones_small['idx'] = range(len(zones_small))
zones_small = zones_small.sort_values('demand', ascending=False).head(20).reset_index(drop=True)
coords = [(pt.y, pt.x) for pt in zones_small.geometry.centroid]
K = len(coords)+1
depot_coord = fac_coords[open_idx]
all_coords = [depot_coord] + coords
D = [[0]*K for _ in range(K)]
for i in range(K):
    for j in range(K):
        if i==j: D[i][j]=0
        else:
            D[i][j]=int(haversine_km(all_coords[i][1], all_coords[i][0], all_coords[j][1], all_coords[j][0])*1000)

demands = [0] + zones_small['demand'].astype(int).tolist()
vehicle_cap = 2000
num_veh = 4
manager = pywrapcp.RoutingIndexManager(K, num_veh, 0)
routing = pywrapcp.RoutingModel(manager)

def dist_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return D[from_node][to_node]
transit_idx = routing.RegisterTransitCallback(dist_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_idx)

def demand_callback(from_index):
    return demands[manager.IndexToNode(from_index)]
demand_idx = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(demand_idx, 0, [vehicle_cap]*num_veh, True, 'Capacity')

search_params = pywrapcp.DefaultRoutingSearchParameters()
search_params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
solution = routing.SolveWithParameters(search_params)
routes = []
if solution:
    for v in range(num_veh):
        idx = routing.Start(v)
        route_nodes = []
        while not routing.IsEnd(idx):
            route_nodes.append(manager.IndexToNode(idx))
            idx = solution.Value(routing.NextVar(idx))
        routes.append(route_nodes)
    print('Routes (node indices incl depot=0):')
    for r in routes:
        print(r)
else:
    print('No VRP solution')

with open('data/vrp_routes.txt','w') as f:
    for r in routes:
        f.write(str(r)+'\n')
print('Saved VRP routes')


## 6) Visualize and save interactive map


In [ ]:
import folium
from folium.plugins import MarkerCluster
cent = [koch.geometry.iloc[0].centroid.y, koch.geometry.iloc[0].centroid.x]
fm = folium.Map(location=cent, zoom_start=12)
mc = MarkerCluster().add_to(fm)
for i,row in cand.iterrows():
    lon = row.geometry.centroid.x
    lat = row.geometry.centroid.y
    folium.Marker([lat, lon], popup=f'Candidate {i}, fixed_cost={row.fixed_cost}').add_to(mc)
for i,o in enumerate(open_list):
    if o==1:
        p = cand.geometry.centroid.iloc[i]
        folium.CircleMarker([p.y,p.x], radius=8, color='green', fill=True, popup='Open Facility').add_to(fm)
fm.save('data/koch_routes_map.html')
print('Saved interactive map to data/koch_routes_map.html')


In [ ]:
import os
print('Saved data files:')
print(os.listdir('data'))
